In [9]:
import random

card_types = ["h", "s", "d", "c"]
cards = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]  # 11 for J, 12 for Q, 13 for K, and 14 for A

class Card():
    def __init__(self, card_type, number):
        self.number = number
        self.card_type = card_type

class Player():
    def __init__(self, name, hand=None, stack=None, points=0):
        if hand is None:
            hand = []
        if stack is None:
            stack = []
        self.name = name
        self.hand = hand
        self.stack = stack
        self.points = points

    def display_hand(self):
        print(f"{self.name}'s hand:")
        for card in self.hand:
            print(f"{card.number}{card.card_type}", end=" ")  # Print each card in the hand
        print()  # Print a newline for readability
    
    def card_list(self):
        return [f"{card.number}{card.card_type}" for card in self.hand]
    
    def remove_card(self, card):
        card_type = card[-1]
        number = int(card[:-1])
        print(f"Removing {card_type} {number}")
        for c in self.hand:
            if c.card_type == card_type and c.number == number:
                self.hand.remove(c)
                return  # Exit the method after removing the card
        print(f"Error: {self.name}'s hand does not contain the card {card}")



class Table():
    def __init__(self, players):
        self.players = players
        self.current_round = 1
        self.current_player_index = 0
        self.last_winner = None
        self.current_trick = []
        self.hearts_broken = False

    def put_card(self, card, player_name):
        # Check if it's the first card in the trick
        if not self.current_trick:
            # First card in the trick
            self.current_trick.append((card, player_name))

            # Check special rules for the first card
            if self.current_round == 1:
                # First round rules
                if card.card_type == "c" and card.number == 2:
                    # Clubs 2 must be put in the first round
                    print(f"{player_name} puts Clubs 2 on the table.")
                    self.current_suit = "c"
                    self.hearts_broken = False
                elif card.card_type == "c":
                    # Clubs 2 wasn't put, invalid move
                    print("Invalid move. In the first round, you must put Clubs 2.")
                elif card.card_type == "h" or (card.card_type == "s" and card.number == 12):
                    # Hearts and Queen of Spades cannot be played in the first round
                    print("Invalid move. In the first round, you cannot play Hearts or Queen of Spades.")
                else:
                    print(f"{player_name} puts {card.number}{card.card_type} on the table.")
                    self.current_suit = card.card_type
            else:
                # Subsequent round rules
                if card.card_type == "h" and not self.hearts_broken:
                    # Hearts cannot be led until broken
                    print("Invalid move. Hearts cannot be led until they are broken.")
                elif card.card_type == "s" and card.number == 12 and not self.hearts_broken:
                    # Queen of Spades cannot be played as the first card
                    print("Invalid move. Queen of Spades cannot be played as the first card.")
                else:
                    print(f"{player_name} puts {card.number}{card.card_type} on the table.")
                    self.current_suit = card.card_type
                    if card.card_type == "h":
                        self.hearts_broken = True
        else:
            # Not the first card in the trick
            current_player = self.players[self.current_player_index]
            if current_player.name == player_name:
                # It's the player's turn to play
                if card.card_type == self.current_suit:
                    # Follow suit rule
                    self.current_trick.append((card, player_name))
                    print(f"{player_name} puts {card.number}{card.card_type} on the table.")
                else:
                    # Violated follow suit rule
                    print("Invalid move. Must follow suit.")
            else:
                # It's not the player's turn to play
                print("Invalid move. It's not your turn to play.")

        if len(self.current_trick) == len(self.players):
            # Determine trick winner
            self.determine_trick_winner()

    def determine_trick_winner(self):
        trick_cards = self.current_trick
        lead_suit_cards = [card for card, _ in trick_cards if card.card_type == self.current_suit]

        if lead_suit_cards:
            # If there are cards in the lead suit, determine the winner based on rank
            trick_winner = max(lead_suit_cards, key=lambda x: x.number)
        else:
            # If no cards in the lead suit, determine the winner based on any suit
            trick_winner = max(trick_cards, key=lambda x: x.number)

        winner_index = [index for index, (_, name) in enumerate(trick_cards) if name == trick_winner.card_type][0]
        winner_name = trick_cards[winner_index][1]

        print(f"{winner_name} takes the trick!")
        self.current_player_index = self.players.index(self.players[winner_index])
        self.players[self.current_player_index].stack.extend([card for card, _ in trick_cards])

        # Update round status after a trick is completed
        self.update_round_status()


    def start_round(self):
        if self.current_round == 1:
            # First round starts with the player having the Club 2
            self.current_player_index = self.find_starting_player_index("c", 2)
        else:
            # Subsequent rounds start with the player who took the last set
            self.current_player_index = self.players.index(self.last_winner)

    def find_starting_player_index(self, card_type, number):
        for i, player in enumerate(self.players):
            if Card(card_type, number) in player.hand:
                return i    

class HeartsGame():
    def __init__(self, players):
        self.table = Table(players)

    def play_game(self):
        while not self.is_game_over():
            # draw the cards
            deck = [Card(card_type, number) for card_type in card_types for number in cards]
            random.shuffle(deck)
            for i, player in enumerate(self.table.players):
                player.hand = deck[i*13:(i+1)*13]
                player.hand.sort(key=lambda x: (x.card_type, x.number))
            
            self.pass_cards()
            self.play_round()

    def play_round(self):
        self.table.start_round()
        for _ in range(len(self.table.players[0].hand)):
            self.play_card()

        # Check if a player attempts to shoot the moon
        for player in self.table.players:
            if self.is_shooting_the_moon(player):
                self.handle_shooting_the_moon(player)

        # Print round results
        self.print_round_results()

    def is_shooting_the_moon(self, player):
        # Check if the player has collected all penalty cards (hearts and Queen of Spades)
        collected_penalty_cards = [card for card in player.stack if card.card_type == "h" or (card.card_type == "s" and card.number == 12)]
        return len(collected_penalty_cards) == 14

    def handle_shooting_the_moon(self, player):
        # Give each opponent penalty points
        for opponent in self.table.players:
            if opponent != player:
                opponent.points += 26  # Each opponent receives 26 penalty points for a successful shoot the moon

    def pass_cards(self):
        # Determine the passing direction based on the current round
        pass_direction = (self.table.current_round) % 4
        pass_list = []

        for i, player in enumerate(self.table.players):
            # Determine the target player to pass cards to
            target_index = (i + pass_direction) % len(self.table.players)
            target_player = self.table.players[target_index]

            # Prompt the current player to select three cards to pass
            passed_cards = self.get_passed_cards(player)
            pass_list.append(passed_cards)
            print(f"{player.name} passes {passed_cards[0]} {passed_cards[1]} {passed_cards[2]} to {target_player.name}.")
        
        for i, player in enumerate(self.table.players):
            target_index = (i + pass_direction) % len(self.table.players)
            target_player = self.table.players[target_index]
            passed_cards = pass_list[i]
            target_player.hand.extend(passed_cards)

    def get_passed_cards(self, player):
        passed_cards = []

        for _ in range(3):
            while True:
                # Prompt the player to input the card they want to pass
                player.display_hand()
                card_input = input(f"Select a card to pass ({_ + 1}/3): ")

                if card_input in player.card_list():
                    passed_cards.append(card_input)
                    player.remove_card(card_input)
                    break
                else:
                    print("You don't have that card in your hand. Please select again.")
        return passed_cards

    def play_card(self):
        current_player = self.table.players[self.table.current_player_index]
        print(f"{current_player.name}'s turn.")
        print(f"Current table: {self.table.card1.number}{self.table.card1.card_type} {self.table.card2.number}{self.table.card2.card_type} {self.table.card3.number}{self.table.card3.card_type} {self.table.card4.number}{self.table.card4.card_type}")

        playable_cards = current_player.get_playable_cards(self.table)

        if not playable_cards:
            # No playable cards, choose any card
            played_card = random.choice(current_player.hand)
        else:
            # Choose a playable card
            played_card = random.choice(playable_cards)

        current_player.hand.remove(played_card)
        self.table.put_card(played_card, current_player.name)

    def get_playable_cards(self, table):
        if table.current_round == 1 and table.current_player_index == 0:
            # First round, first player, must play Clubs 2
            return [card for card in self.hand if card.card_type == "c" and card.number == 2]
        elif table.current_round == 1 and any(card.card_type == "c" for card in table.players[0].stack):
            # First round, someone played Clubs, play anything except Hearts or Queen of Spades
            return [card for card in self.hand if card.card_type != "h" and card.number != 12]
        elif table.current_round > 1 and table.players[0].stack:
            # Hearts are unlocked, play anything
            return self.hand
        elif table.current_round > 1 and not table.players[0].stack:
            # Hearts are not unlocked, play anything except Hearts or Queen of Spades
            return [card for card in self.hand if card.card_type != "h" and card.number != 12]
        else:
            # Default case, play anything
            return self.hand

    def is_game_over(self):
        points = [player.points for player in self.table.players]
        points.sort()
        if points[0] >= 100:
            if points[-1] != points[-2]:
                print(f"Game over! {self.table.players[0].name} won!", "With points: ", points)
                return True
            return False
        
    def print_round_results(self):
        print("Round Results:")
        for player in self.table.players:
            print(f"{player.name}: {player.points} points")
        print()

# Example usage:
player1 = Player("Player 1")
player2 = Player("Player 2")
player3 = Player("Player 3")
player4 = Player("Player 4")

players = [player1, player2, player3, player4]

hearts_game = HeartsGame(players)
hearts_game.play_game()


Player 1's hand:
6c 10c 11c 12c 2d 8d 10d 13h 14h 4s 6s 10s 11s 
Removing h 14
Player 1's hand:
6c 10c 11c 12c 2d 8d 10d 13h 4s 6s 10s 11s 
Removing h 13
Player 1's hand:
6c 10c 11c 12c 2d 8d 10d 4s 6s 10s 11s 
Removing s 11
Player 1 passes 14h 13h 11s to Player 2.
Player 2's hand:
3c 5c 7d 11d 13d 3h 4h 8h 11h 7s 9s 12s 14s 
Removing s 14
Player 2's hand:
3c 5c 7d 11d 13d 3h 4h 8h 11h 7s 9s 12s 
Removing s 12
Player 2's hand:
3c 5c 7d 11d 13d 3h 4h 8h 11h 7s 9s 
Removing h 11
Player 2 passes 14s 12s 11h to Player 3.
Player 3's hand:
7c 13c 14c 4d 9d 12d 14d 5h 6h 10h 12h 3s 5s 
Removing h 12
Player 3's hand:
7c 13c 14c 4d 9d 12d 14d 5h 6h 10h 3s 5s 
Removing h 10
Player 3's hand:
7c 13c 14c 4d 9d 12d 14d 5h 6h 3s 5s 
Removing d 14
Player 3 passes 12h 10h 14d to Player 4.
Player 4's hand:
2c 4c 8c 9c 3d 5d 6d 2h 7h 9h 2s 8s 13s 
Removing s 13
Player 4's hand:
2c 4c 8c 9c 3d 5d 6d 2h 7h 9h 2s 8s 
Removing h 9
Player 4's hand:
2c 4c 8c 9c 3d 5d 6d 2h 7h 2s 8s 
Removing c 2
Player 4 passe

TypeError: list indices must be integers or slices, not NoneType